I200635
MAHNOOR AKHTAR
SECTION A CS

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/TEXT_TO_SPEECH/Data/STT_Recordings')


In [3]:

!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.8 MB/s eta 0:00:00


In [4]:
!pip install transformers
!pip install torchaudio
!pip install datasets
!pip install torch torchaudio transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [5]:
import os
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torchaudio
from torchaudio.transforms import Resample
import pandas as pd


In [ ]:
import os
import pandas as pd

# Step 2: List all audio files in the "SST_Recording" folder
audio_folder = "/content/drive/MyDrive/TEXT_TO_SPEECH/Data/STT_Recordings"
audio_files = os.listdir(audio_folder)

# Step 3: Read the combined CSV file
csv_file = "/content/drive/MyDrive/TEXT_TO_SPEECH/Data/STT_Recordings/Combined.csv"
df = pd.read_csv(csv_file)

# Step 4: Filter the data based on your criteria
def is_valid_audio(audio_file):
    if pd.notna(audio_file) and audio_file.endswith(".wav") and audio_file in audio_files:
        audio_path = os.path.join(audio_folder, audio_file)
        return os.path.getsize(audio_path) > 0  # Check if the audio file is not empty
    return False

def has_valid_transcript(row):
    transcript = row['Transcript']
    return pd.notna(transcript) and not (str(transcript).strip().isdigit())

df['ValidAudio'] = df['Audio File'].apply(is_valid_audio)
df = df[df['ValidAudio'] & df.apply(has_valid_transcript, axis=1)]

# Step 5: Create a new cleaned CSV file
cleaned_csv_file = "cleaned_combined.csv"
df.to_csv(cleaned_csv_file, index=False)

print("Data cleaning completed. Cleaned CSV saved as 'cleaned_combined.csv'.")


Data cleaning completed. Cleaned CSV saved as 'cleaned_combined.csv'.


In [ ]:
import pandas as pd

# Step 1: Read the cleaned CSV file
cleaned_csv_file = "/content/drive/MyDrive/TEXT_TO_SPEECH/Data/STT_Recordings/cleaned_combined.csv"
df = pd.read_csv(cleaned_csv_file)

# Step 2: Remove rows with numeric transcript values (numbers in character format)
df = df[~df['Transcript'].str.strip().str.match(r'^\d+$')]

# Step 3: Create a new CSV file without numeric transcript rows
final_cleaned_csv_file = "STT.csv"
df.to_csv(final_cleaned_csv_file, index=False)

print("Numeric transcript rows removed. Final cleaned CSV saved as 'STT.csv'.")


Numeric transcript rows removed. Final cleaned CSV saved as 'STT.csv'.


In [ ]:
import pandas as pd

# Step 1: Read the original CSV file
original_csv_file = "STT.csv"
df = pd.read_csv(original_csv_file)

# Step 2: Drop the specified columns
columns_to_remove = ["Unnamed: 2", "ValidAudio"]
df = df.drop(columns=columns_to_remove, errors="ignore")

# Step 3: Create a new CSV file without the specified columns
new_csv_file = "STT_no_columns.csv"
df.to_csv(new_csv_file, index=False)

print(f"Columns {', '.join(columns_to_remove)} removed. New CSV saved as '{new_csv_file}'.")


Columns Unnamed: 2, ValidAudio removed. New CSV saved as 'STT.csv'.


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Load the dataset from STT_no_columns.csv
dataset_file = "STT_no_columns.csv"
df = pd.read_csv(dataset_file)

# Step 2: Define your features (X) and target (y)
# In your case, if you don't have a target column and you want to split the dataset randomly, you can do:
X = df

# Step 3: Split the dataset into training and testing sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Step 4: X_train and X_test now contain the training and testing datasets, respectively
print(f"Training dataset shape: {X_train.shape}")
print(f"Testing dataset shape: {X_test.shape}")


Training dataset shape: (6131, 2)
Testing dataset shape: (1533, 2)


In [15]:
import json

custom_vocab_path = "/content/drive/MyDrive/TEXT_TO_SPEECH/vocab.json"

with open(custom_vocab_path, "r", encoding="utf-8") as json_file:
    custom_vocab = json.load(json_file)

if ' ' not in custom_vocab:
    custom_vocab[' '] = len(custom_vocab)

# Check and add the [PAD] token if it doesn't exist
if "[PAD]" not in custom_vocab:
    custom_vocab["[PAD]"] = len(custom_vocab)

In [23]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="ur", task="transcribe")

from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [11]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./whisper-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=5_000,
    save_strategy="steps",
    logging_dir="./logs",
)

In [12]:
# Define the data preprocessing function
def prepare_dataset(batch):
    inputs = tokenizer(batch["Audio File"], return_tensors="pt", padding="longest", truncation=True)
    inputs["labels"] = tokenizer(batch["Transcript"], return_tensors="pt", padding="longest", truncation=True).input_ids
    return inputs

In [24]:
# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=prepare_dataset,
    train_dataset=X_train,
    eval_dataset=X_test,
)

In [31]:
trainer.train()


KeyError: ignored

In [29]:
row_4999 = X_train.iloc[6130]


In [30]:
print(row_4999)

Audio File     097f093a-8da8-456d-bee2-35e61af67983.wav
Transcript    انہوں نے کہا جی یہ کر لو میں نے کہا یہ تو
Name: 7270, dtype: object
